In [1]:
%%writefile 2.3-convolve-1d-shared-constant-memory.py

import math
import numpy as np
from numba import cuda, float32

thread_block_size_deafult = 256

h_host = np.random.rand(256).astype(np.float32)


@cuda.jit
def convolve_kernel(y, x):
    i = cuda.blockIdx.x*cuda.blockDim.x + cuda.threadIdx.x
    
    if i >= y.shape[0]:
        return
    
    # Constant memory.
    h_gpu_const = cuda.const.array_like(h_host)
    filter_size = len(h_gpu_const)
    
    # Shared memory.
    x_sm = cuda.shared.array(shape=0, dtype=float32)
    sm_size = cuda.blockDim.x+filter_size-1
    
    # Copy a portion of global memory data to shared memory.
    k = i - (filter_size-1) # The current position in the global memory.
    k_sm = cuda.threadIdx.x # The current position in the shared memory.  
    while k_sm < sm_size:
        if k < 0:
            x_sm[k_sm] = float32(0.0)
        else:
            x_sm[k_sm] = x[k]
        k_sm += cuda.blockDim.x
        k    += cuda.blockDim.x

    cuda.syncthreads()
    
    k_sm = cuda.threadIdx.x+filter_size-1
    value = float32(0.0)
    for j in range(filter_size):
        value += x_sm[k_sm-j]*h_gpu_const[j]
    y[i] = value

    
def convolve(y, x):
    thread_block_size = min(thread_block_size_deafult, len(y))
    block_size = (thread_block_size, )
    grid_size = (math.ceil(len(y)/block_size[0]), )
    filter_size = len(h_host)
    shared_memory_size = thread_block_size+filter_size-1
    shared_memory_size_bytes = shared_memory_size*y.dtype.itemsize
    convolve_kernel[grid_size, block_size, cuda.default_stream(), shared_memory_size_bytes](y, x)  
    
# Test data.
n = 1024*64*16*16

for i in range(20):
    x_host = np.random.rand(n).astype(np.float32)
    y_gpu = cuda.device_array(shape=(n,), dtype=np.float32)
    x_gpu = cuda.to_device(x_host)
    convolve(y_gpu, x_gpu)
    y_host = y_gpu.copy_to_host()

Overwriting 2.3-convolve-1d-shared-constant-memory.py


In [2]:
! nsys profile --stats=true -t cuda python 2.3-convolve-1d-shared-constant-memory.py

Processing events...
Saving temporary "/tmp/nsys-report-c57f-2788-01ed-8e2c.qdstrm" file to disk...
Creating final output files...

Processing [==============================================================100%]
Saved report file to "/tmp/nsys-report-c57f-2788-01ed-8e2c.qdrep"
Exporting 253 events: [===================================================100%]

Exported successfully to
/tmp/nsys-report-c57f-2788-01ed-8e2c.sqlite

Generating CUDA API Statistics...
CUDA API Statistics (nanoseconds)

Time(%)      Total Time       Calls         Average         Minimum         Maximum  Name                                                                            
-------  --------------  ----------  --------------  --------------  --------------  --------------------------------------------------------------------------------
   77.4       629457407          20      31472870.4        30758890        32572691  cuMemcpyDtoH_v2                                                                 
   1

In [4]:
! ncu python 2.2-convolve-1d-shared-memory.py

==PROF== Connected to process 13842 (/opt/conda/envs/rapids/bin/python3.7)
==PROF== Profiling "convolve_kernel$241" - 1: 0%....50%....100% - 8 passes
==PROF== Profiling "convolve_kernel$241" - 2: 0%....50%....100% - 8 passes
==PROF== Profiling "convolve_kernel$241" - 3: 0%....50%....100% - 8 passes
==PROF== Profiling "convolve_kernel$241" - 4: 0%....50%....100% - 8 passes
==PROF== Profiling "convolve_kernel$241" - 5: 0%....50%....100% - 8 passes
==PROF== Profiling "convolve_kernel$241" - 6: 0%....50%....100% - 8 passes
==PROF== Profiling "convolve_kernel$241" - 7: 0%....50%....100% - 8 passes
==PROF== Profiling "convolve_kernel$241" - 8: 0%....50%....100% - 8 passes
==PROF== Profiling "convolve_kernel$241" - 9: 0%....50%....100% - 8 passes
==PROF== Profiling "convolve_kernel$241" - 10: 0%....50%....100% - 8 passes
==PROF== Profiling "convolve_kernel$241" - 11: 0%....50%....100% - 8 passes
==PROF== Profiling "convolve_kernel$241" - 12: 0%....50%....100% - 8 passes
==PROF== Profiling "co